In [1]:
# Imports e instalações

import pandas as pd
import numpy as np

# Viz
import matplotlib.pyplot as plt
import seaborn as sns

#!pip install scikit-learn
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier

# Métricas
from sklearn.metrics import (accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix, classification_report, f1_score)
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay

In [5]:
df = pd.read_csv(r'C:\Users\JacyzinGuilherme(Bip\mentoria-bip\dados_editados\australia_clima_v0.csv', sep=',')


In [6]:
df

,data,localidade,temp_min,temp_max,chuva_mm,evaporacao,horas_sol,vento_rajada_direcao,vento_rajada_velocidade,vento_direcao_9h,...,co2_media_5anos_x,co2_emissao_y,co2_norm_y,co2_delta_y,co2_media_5anos_y,estacao,altitude,distancia_costa,ponto_orvalho_15h,depressao_orvalho_15h
0,2007-11-01,Canberra,8.0,24.3,0.0,3.4,6.3,NW,30.0,SW,...,613.78370,630.7811,0.915342,-17.6520,613.78370,primavera,575.0,247.882185,9.4,14.2
1,2007-11-02,Canberra,14.0,26.9,3.6,4.4,9.7,ENE,39.0,E,...,613.78370,630.7811,0.915342,-17.6520,613.78370,primavera,575.0,247.882185,12.9,12.8
2,2007-11-03,Canberra,13.7,23.4,3.6,5.8,3.3,NW,85.0,N,...,613.78370,630.7811,0.915342,-17.6520,613.78370,primavera,575.0,247.882185,14.0,6.2
3,2007-11-04,Canberra,13.3,15.5,39.8,7.2,9.1,NW,54.0,WNW,...,613.78370,630.7811,0.915342,-17.6520,613.78370,primavera,575.0,247.882185,5.3,8.8
4,2007-11-05,Canberra,7.6,16.1,2.8,5.6,10.6,SSE,50.0,SSE,...,613.78370,630.7811,0.915342,-17.6520,613.78370,primavera,575.0,247.882185,5.2,10.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154666,2017-06-25,Melbourne,5.5,13.9,0.0,2.8,3.8,NNW,44.0,N,...,536.29274,536.7209,0.464232,39.8858,536.29274,inverno,31.0,0.540212,3.8,8.4
154667,2017-06-25,Melbourne,8.6,14.3,0.0,2.8,3.8,NW,35.0,N,...,536.29274,536.7209,0.464232,39.8858,536.29274,inverno,31.0,0.540212,4.6,8.8
154668,2017-06-25,Perth,6.3,17.0,0.0,1.6,7.9,E,26.0,SE,...,536.29274,536.7209,0.464232,39.8858,536.29274,inverno,15.0,0.656682,5.4,10.2
154669,2017-06-25,Perth,6.3,17.0,0.0,1.6,7.9,E,26.0,SE,...,536.29274,536.7209,0.464232,39.8858,536.29274,inverno,15.0,458.810201,5.4,10.2


In [7]:
df.isna().sum()

data                            0
localidade                      0
temp_min                        0
temp_max                        0
chuva_mm                        0
evaporacao                      0
horas_sol                       0
vento_rajada_direcao            0
vento_rajada_velocidade         0
vento_direcao_9h                0
vento_direcao_15h               0
vento_veloc_9h                  0
vento_veloc_15h                 0
umidade_9h                      0
umidade_15h                     0
pressao_9h                      0
pressao_15h                     0
nuvem_9h                        0
nuvem_15h                       0
temp_9h                         0
temp_15h                        0
choveu_hoje                     0
chove_amanha                    0
ano                             0
mes                             0
dia                             0
trimestre                       0
mes_sin                         0
mes_cos                         0
amplitude_term

In [8]:
# converte a coluna "data" para o tipo datetime
# dayfirst=True garante que datas como 13/12/2008 sejam interpretadas como dia/mês/ano
df["data"] = pd.to_datetime(df["data"], dayfirst=True, errors="coerce")


In [9]:
# ordena o dataframe por data (do passado para o futuro)
# isso é obrigatório para fazer split temporal sem vazamento
df = df.sort_values("data")


In [10]:
# confere se ainda existe algum valor nulo na coluna data após a conversão
# o ideal é retornar 0
df["data"].isna().sum()


np.int64(93603)

In [11]:
# cria uma lista com todas as colunas que terminam com "_x"
# isso identifica colunas duplicadas geradas por merge
colunas_x = [c for c in df.columns if c.endswith("_x")]


In [12]:
# cria uma lista com todas as colunas que terminam com "_y"
# isso identifica a outra versão duplicada gerada por merge
colunas_y = [c for c in df.columns if c.endswith("_y")]


In [13]:
# mostra as colunas _x e _y encontradas
# isso permite você visualizar rapidamente quais duplicatas existem
colunas_x, colunas_y


(['co2_emissao_x', 'co2_norm_x', 'co2_delta_x', 'co2_media_5anos_x'],
 ['co2_emissao_y', 'co2_norm_y', 'co2_delta_y', 'co2_media_5anos_y'])

In [14]:
# percorre cada coluna que termina com "_x"
for col_x in colunas_x:
    
    # cria o nome esperado da coluna correspondente "_y"
    col_y = col_x.replace("_x", "_y")
    
    # verifica se a coluna "_y" realmente existe no dataframe
    if col_y in df.columns:
        
        # calcula quantos nulos existem na coluna _x
        nulos_x = df[col_x].isna().sum()
        
        # calcula quantos nulos existem na coluna _y
        nulos_y = df[col_y].isna().sum()
        
        # se a coluna _x tiver menos ou igual nulos do que a _y, mantemos _x e removemos _y
        if nulos_x <= nulos_y:
            # remove a coluna _y, pois ela é a versão “pior” (mais nulos)
            df = df.drop(columns=[col_y])
            
            # renomeia a coluna _x para o nome limpo sem sufixo
            df = df.rename(columns={col_x: col_x.replace("_x", "")})
        
        # senão, mantemos _y e removemos _x
        else:
            # remove a coluna _x, pois ela é a versão “pior” (mais nulos)
            df = df.drop(columns=[col_x])
            
            # renomeia a coluna _y para o nome limpo sem sufixo
            df = df.rename(columns={col_y: col_y.replace("_y", "")})


In [15]:
# atualiza a lista de colunas _x depois das remoções anteriores
colunas_x_restantes = [c for c in df.columns if c.endswith("_x")]


In [16]:
# percorre cada coluna _x que sobrou sozinha
for col_x in colunas_x_restantes:
    
    # renomeia a coluna removendo o sufixo _x
    df = df.rename(columns={col_x: col_x.replace("_x", "")})


In [17]:
# verifica se ainda existe alguma coluna que termina com "_x" ou "_y"
# o ideal é retornar listas vazias
[c for c in df.columns if c.endswith("_x")], [c for c in df.columns if c.endswith("_y")]


([], [])

In [19]:
# define a coluna alvo (target) como "chove_amanha"
# esse y será o que o modelo tentará prever
y = df["chove_amanha"]


In [20]:
# define uma lista de colunas que serão removidas antes de criar X
# essas colunas não devem ser usadas como variáveis explicativas no baseline
colunas_para_remover_do_X = ["chove_amanha", "data", "localidade"]

In [21]:
# cria o dataframe X removendo as colunas que não devem entrar no modelo
X = df.drop(columns=colunas_para_remover_do_X, errors="ignore")


In [22]:
# confere quantas colunas ficaram em X
# isso mostra o “tamanho” do conjunto de features
X.shape


(154671, 53)

In [23]:
# calcula o índice de corte para separar 80% treino e 20% teste
indice_corte = int(len(df) * 0.8)


In [24]:
# cria X_train pegando as linhas antigas (início do dataset)
X_train = X.iloc[:indice_corte]


In [25]:
# cria y_train pegando as linhas antigas do target
y_train = y.iloc[:indice_corte]


In [26]:
# cria X_test pegando as linhas mais novas (final do dataset)
X_test = X.iloc[indice_corte:]


In [27]:
# cria y_test pegando as linhas mais novas do target
y_test = y.iloc[indice_corte:]


In [28]:
# imprime as datas mínima e máxima do treino e do teste
# isso confirma que o split é temporal e não aleatório
print("Treino:", df["data"].iloc[:indice_corte].min(), "->", df["data"].iloc[:indice_corte].max())
print("Teste :", df["data"].iloc[indice_corte:].min(), "->", df["data"].iloc[indice_corte:].max())


Treino: 2007-01-11 00:00:00 -> 2017-12-06 00:00:00
Teste : NaT -> NaT


In [30]:
# identifica colunas categóricas (tipo object) dentro de X_train
# essas colunas precisam de OneHotEncoder
colunas_categoricas = X_train.select_dtypes(include=["object"]).columns.tolist()


In [31]:
# identifica colunas numéricas dentro de X_train
# essas colunas recebem imputação e padronização
colunas_numericas = X_train.select_dtypes(exclude=["object"]).columns.tolist()


In [32]:
# imprime quantas colunas categóricas e numéricas existem
# isso ajuda a validar se o dataset está bem tipado
print("Categóricas:", len(colunas_categoricas), colunas_categoricas)
print("Numéricas  :", len(colunas_numericas))


Categóricas: 6 ['vento_rajada_direcao', 'vento_direcao_9h', 'vento_direcao_15h', 'fase_enso', 'intensidade_enso', 'estacao']
Numéricas  : 47


In [33]:
# cria um pipeline para colunas numéricas
# primeiro preenche nulos com mediana (robustez)
# depois padroniza as escalas (ajuda a regressão logística)
transformador_numerico = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])


In [36]:
# cria um pipeline para colunas categóricas
# primeiro preenche nulos com a categoria mais frequente (robustez)
# depois aplica one-hot encoding ignorando categorias desconhecidas
transformador_categorico = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])


In [37]:
# cria o preprocessador que aplica transformador numérico nas colunas numéricas
# e aplica transformador categórico nas colunas categóricas
preprocessador = ColumnTransformer(transformers=[
    ("num", transformador_numerico, colunas_numericas),
    ("cat", transformador_categorico, colunas_categoricas)
])


In [53]:
# cria o modelo de regressão logística
# class_weight="balanced" ajuda quando a classe "chove" é minoritária
# max_iter alto evita erro de convergência
modelo = LogisticRegression(max_iter=2000, class_weight="balanced", random_state=42)


In [54]:
# cria o pipeline completo com preprocessamento e modelo
pipeline = Pipeline(steps=[
    ("preprocessador", preprocessador),
    ("modelo", modelo)
])


In [55]:
# treina o pipeline usando o conjunto de treino
pipeline.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessador', ...), ('modelo', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers

In [56]:
# gera a previsão de classe (0 ou 1) no conjunto de teste
y_pred = pipeline.predict(X_test)



In [57]:
# gera a probabilidade da classe 1 (chuva) para calcular ROC-AUC
y_proba = pipeline.predict_proba(X_test)[:, 1]
# gera a previsão de probabilidades no conjunto de teste

In [58]:
# calcula métricas principais
acc = accuracy_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
pre = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)


In [59]:
# imprime as métricas de forma organizada
print("Accuracy :", acc)
print("Recall   :", rec)
print("Precision:", pre)
print("F1       :", f1)
print("ROC-AUC  :", auc)


Accuracy : 0.7892678196217876
Recall   : 0.7303149606299213
Precision: 0.5044451417215773
F1       : 0.5967213114754099
ROC-AUC  : 0.8498905576958365


In [60]:
# calcula a matriz de confusão para entender erros do modelo
matriz = confusion_matrix(y_test, y_pred)


In [61]:
# imprime a matriz de confusão no formato bruto
# depois a gente interpreta TP, FP, FN, TN
print(matriz)


[[19593  4738]
 [ 1781  4823]]


In [62]:
# extrai TN, FP, FN, TP da matriz de confusão
tn, fp, fn, tp = matriz.ravel()

# imprime cada componente com significado claro
print("TN (previu NAO e NAO choveu):", tn)
print("FP (previu SIM mas NAO choveu):", fp)
print("FN (previu NAO mas CHOVEU):", fn)
print("TP (previu SIM e CHOVEU):", tp)


TN (previu NAO e NAO choveu): 19593
FP (previu SIM mas NAO choveu): 4738
FN (previu NAO mas CHOVEU): 1781
TP (previu SIM e CHOVEU): 4823


In [64]:
# cria o modelo Random Forest
# n_estimators define quantas árvores serão criadas
# max_depth limita a profundidade das árvores para evitar overfitting
# min_samples_leaf garante que folhas tenham volume mínimo de dados
# class_weight="balanced" ajuda quando a classe chuva é minoritária
rf_modelo = RandomForestClassifier(
    n_estimators=300,
    max_depth=15,
    min_samples_leaf=50,
    random_state=42,
    n_jobs=-1,
    class_weight="balanced"
)


In [65]:
# cria um transformador numérico específico para árvores
# apenas imputa valores nulos, sem padronizar escala
transformador_numerico_rf = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])


In [66]:
# cria o preprocessador específico para Random Forest
preprocessador_rf = ColumnTransformer(transformers=[
    ("num", transformador_numerico_rf, colunas_numericas),
    ("cat", transformador_categorico, colunas_categoricas)
])


In [67]:
# cria o pipeline completo com preprocessamento e Random Forest
pipeline_rf = Pipeline(steps=[
    ("preprocessador", preprocessador_rf),
    ("modelo", rf_modelo)
])


In [69]:
# treina o pipeline Random Forest usando o conjunto de treino
pipeline_rf.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessador', ...), ('modelo', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers

In [70]:
# gera previsões de classe no conjunto de teste
y_pred_rf = pipeline_rf.predict(X_test)

# gera probabilidades da classe chuva
y_proba_rf = pipeline_rf.predict_proba(X_test)[:, 1]



In [71]:
# calcula métricas do Random Forest
acc_rf = accuracy_score(y_test, y_pred_rf)
rec_rf = recall_score(y_test, y_pred_rf)
pre_rf = precision_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)
auc_rf = roc_auc_score(y_test, y_proba_rf)


In [73]:
# imprime métricas do Random Forest
print("RANDOM FOREST")
print("Accuracy :", acc_rf)
print("Recall   :", rec_rf)
print("Precision:", pre_rf)
print("F1       :", f1_rf)
print("ROC-AUC  :", auc_rf)


RANDOM FOREST
Accuracy : 0.8118635849361564
Recall   : 0.7144155057540884
Precision: 0.5453074433656958
F1       : 0.6185107498689041
ROC-AUC  : 0.8659579935077202


In [74]:
# calcula a matriz de confusão do Random Forest
matriz_rf = confusion_matrix(y_test, y_pred_rf)
print(matriz_rf)


[[20397  3934]
 [ 1886  4718]]


In [75]:
# importa o classificador XGBoost
# ele usa boosting sequencial para corrigir erros das árvores anteriores
from xgboost import XGBClassifier


In [98]:
# cria o modelo XGBoost
# n_estimators define quantas árvores serão criadas
# max_depth controla complexidade
# learning_rate controla o quanto cada árvore contribui
# subsample e colsample_bytree reduzem overfitting
xgb_modelo = XGBClassifier(
    n_estimators=2000,
    learning_rate=0.03,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    objective="binary:logistic",
    eval_metric="aucpr",
    scale_pos_weight=5.0,
    #scale_pos_weight=scale_pos_weight,
    random_state=42,
    #reg_lambda=1.0,
    reg_alpha=0.8,
    tree_method='hist',
    enable_categorical=True,
    n_jobs=-1
)


In [99]:
# cria o pipeline completo com preprocessamento e XGBoost
pipeline_xgb = Pipeline(steps=[
    ("preprocessador", preprocessador_rf),
    ("modelo", xgb_modelo)
])


In [100]:
# treina o pipeline XGBoost
pipeline_xgb.fit(X_train, y_train)
# gera previsões no conjunto de teste
y_pred_xgb = pipeline_xgb.predict(X_test)
# gera probabilidades da classe chuva
y_proba_xgb = pipeline_xgb.predict_proba(X_test)[:, 1]



In [101]:
# calcula métricas do XGBoost
acc_xgb = accuracy_score(y_test, y_pred_xgb)
rec_xgb = recall_score(y_test, y_pred_xgb)
pre_xgb = precision_score(y_test, y_pred_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb)
auc_xgb = roc_auc_score(y_test, y_proba_xgb)


In [102]:
# imprime métricas do XGBoost
print("XGBOOST")
print("Accuracy :", acc_xgb)
print("Recall   :", rec_xgb)
print("Precision:", pre_xgb)
print("F1       :", f1_xgb)
print("ROC-AUC  :", auc_xgb)


XGBOOST
Accuracy : 0.8074995959269435
Recall   : 0.7839188370684433
Precision: 0.5334363730036064
F1       : 0.6348641854190937
ROC-AUC  : 0.8870780013811633


In [103]:
# matriz de confusão do XGBoost
matriz_xgb = confusion_matrix(y_test, y_pred_xgb)
print(matriz_xgb)


[[19803  4528]
 [ 1427  5177]]
